## Periodic 3-D grid


Each node has 6 links to its four nearest neighbours. X axis points in the horizontal direction, Y in the vertical. 

Coin: Grover or Fourier

Phase randomization: optional

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats as st
########################################## parameters
N = 729
Nx = np.cbrt(N).astype(int) 
Ny = np.cbrt(N).astype(int) 
Nz = np.cbrt(N).astype(int)

W = 0.0
grover6 = np.ones((6,6))
for i in range(0,6):
    grover6[i,i] = -2
grover6 /= 3
#print(grover)
fourier6 = np.ones((6,6), dtype=np.complex128)
for i in range(1,6):
    for j in range(1,6):
        fourier6[i][j] = np.exp(2j*np.pi*i*j/6)
fourier6 /= np.sqrt(6)
#print(fourier)

In [2]:
def plot_degree_dist(G):
    degs = np.array([4,5,6,7,8])
    bins = [3.5,4.5,5.5,6.5,7.5,8.5]
    degrees = np.array([G.degree(n) for n in G.nodes()])
    plt.figure(3)
    counts, edges, bars = plt.hist(degrees, bins=bins, density=True)
    plt.bar_label(bars)
    plt.title("Degree distribution")
    plt.xlabel("degree")
    plt.ylabel("counts")
    plt.show()
    
def make_graph():
    G = nx.Graph()
    for k in range(0,Nz):
        for j in range(0,Ny):
            for i in range(0,Nx):
                G.add_node(i + Nx*j + Nx*Ny*k, pos=(i,j,k))

    for k in range(0,Nz):           
        for j in range(0,Ny):
            for i in range(0,Nx):
                    G.add_edge( i + Nx*j + Nx*Ny*k, i + Nx*((j+1)%Ny) + Nx*Ny*k)
                    G.add_edge( i + Nx*j + Nx*Ny*k, ((i+1)%Nx) + Nx*j + Nx*Ny*k)
                    G.add_edge(i + Nx*j + Nx*Ny*k, i + Nx*j + Nx*Ny*((k+1)%Nz))
    #print(list(G.nodes))
    plt.figure(1)
    nx.draw(G, with_labels=True, edgelist = G.edges)
    plt.show()
    plt.figure(2)
    nx.draw_shell(G,  with_labels=True)
    plt.show()
    #plot_degree_dist(G)

In [3]:
###################################################################################### functions ###
def S(state):
    for k in range(0,Nz):
        for j in range(0,Ny):
            for i in range(0,Nx):
                    state[i,j,k,1], state[(i+1)%Nx,j,k,3] = state[(i+1)%Nx,j,k,3], state[i,j,k,1]
                    state[i,j,k,0], state[i,(j+1)%Ny,k,2] = state[i,(j+1)%Ny,k,2], state[i,j,k,0]
                    state[i,j,k,4], state[i,j,(k+1)%Nz,5] = state[i,j,(k+1)%Nz,5], state[i,j,k,4]
    #print("state after S", state)
    return state

def C_Grover(state):
    for k in range(0,Nz):
        for j in range(0,Ny):
            for i in range(0,Nx):
                state[i,j,k,0:6] = np.matmul(grover6,state[i,j,k,0:6])
    #print("state after C", state)
    return state

def C_Fourier(state):
    for k in range(0,Nz):
        for j in range(0,Ny):
            for i in range(0,Nx):
                state[i,j,k,0:6] = np.matmul(fourier6,state[i,j,k,0:6])
    #print("state after C", state)
    return state

def phase(state,w):
    dim = (state.shape[0], state.shape[1],state.shape[2])
    #print(dim)
    for l in range(0,6):
        rnd = np.random.uniform(-w,w,size=dim)
        state[:,:,:,l] = state[:,:,:,l]*np.exp(2*np.pi*1j*rnd)
    return state

########################################################### to check normalization, plus to get amplitudes of the state at various position n
def magn(state):
    magnit = np.zeros((Nx,Ny,Nz))
    #check = 0
    for k in range(0,Nz):
        for j in range(0,Ny):
            for i in range(0,Nx):
                for l in range(0,6):
                        magnit[i,j,k] += state[i,j,k,l].conjugate()*state[i,j,k,l]
    #print("check value", check)
    return magnit

In [13]:
def main(w, runtime):
    # ################################# state declaration and initialization
    state = np.zeros((Nx,Ny,Nz,6), dtype=np.complex128)  
    ################################### state initialized at the middle point of the lattice
    # state[int(Nx/2), int(Ny/2), int(Nz/2), 0:6] = 1/np.sqrt(6)
    ################################### state initialized at a random point of the lattice
    I,J,K = np.random.randint(0,Nx), np.random.randint(0,Ny), np.random.randint(0,Nz)
    print("initial position", I,J,K)
    state[I,J,K, 0:6] = 1/np.sqrt(6)
    #uncomment the following to have a uniform superposition instead
    # state[:,:,:,0:6].fill(1/np.sqrt(6*N))  
    
    # make_graph()
    
    ####################### magnitude stores wave amplitude at each position for all timestamps
    magnitude = np.zeros((Nx,Ny,Nz,runtime))
    magnitude[:,:,:,0] = magn(state)
    
    ######################## run the walker
    for j in range(1,runtime):
        # state = S(C_Grover(phase(state,w)))
        # state = S(C_Fourier(phase(state,w)))
        state = S(C_Fourier(state))
        magnitude[:,:,:,j] = magn(state)

    return magnitude

In [14]:
def draw_timeslices(magnitude, run, w):
    z_coo = np.random.randint(0,Nz,1)
    fig, ax = plt.subplots(2,2, figsize=(15,10))
    fig.suptitle("for Graph run: {}, W = {}".format(run, w))
    im = ax[0,0].matshow(magnitude[:,:,z_coo,runtime-1])
    ax[0,0].set_title('time = {}'.format(runtime))
    plt.colorbar(im, ax=ax[0,0])
    
    im = ax[0,1].matshow(magnitude[:,:,z_coo,int(runtime*0.75)])
    ax[0,1].set_title('time = {}'.format(int(runtime*0.75)))
    plt.colorbar(im, ax=ax[0,1])
    
    im = ax[1,0].matshow(magnitude[:,:,z_coo,int(runtime*0.5)])
    ax[1,0].set_title('time = {}'.format(int(runtime*0.5)))
    plt.colorbar(im, ax=ax[1,0])
    
    im = ax[1,1].matshow(magnitude[:,:,z_coo,int(runtime*0.25)])
    ax[1,1].set_title('time = {}'.format(int(runtime*0.25)))
    plt.colorbar(im, ax=ax[1,1])
    plt.show()

In [15]:
def draw_timeseries(magnitude,I,J,K,stab_time,runtime,run,w):
    time = np.arange(stab_time, runtime)
    plt.figure(100+run)
    fig, ax = plt.subplots(1,2, figsize=(12,2.5))
    ax[0].plot(time, magnitude[stab_time:])
    ax[0].set_title("for N = {}, W = {}".format(N,w))
    ax[0].set_xlabel("time")
    ax[0].set_ylabel("probability amplitude at ({},{},{})".format(I,J,K))
    
    ax[1].hist(magnitude[stab_time:], bins=100, density=True, log=True, histtype='step')
    ax[1].set_title("for Graph run: {}".format(run))
    ax[1].set_xlabel("probability amplitude at ({},{},{})".format(I,J,K))
    ax[1].set_ylabel("Probability density")
    plt.show()

In [16]:
def EE_analysis(means, stds, runtime, stab_time, w):
    count = 0
    P_th = means + 3*stds
    print(P_th.shape)
    for k in range(0,Nz):
        for j in range(0,Ny):
            for i in range(0,Nx):
                for time in range(stab_time,runtime):
                    if (magnitude[i,j,k,time] > P_th[i,j,k]):
                        count += 1
    perc = 100*count/(N*(runtime-stab_time))
    print("rouge event count for w = {} is: ".format(w), count,\
          " out of total runtime*N events, that is {} %".format(perc))
    return perc

In [17]:
RUN = 10
means = np.zeros((Nx, Ny, Nz))
stds = np.zeros((Nx, Ny, Nz))
means_run = np.zeros((Nx, Ny, Nz, RUN))
stds_run = np.zeros((Nx, Ny, Nz, RUN))
means = np.zeros((Nx, Ny, Nz))
stds = np.zeros((Nx, Ny, Nz))
expected = 6/(6*N)
perc_ee = np.zeros(RUN)

for run in range(0,RUN):
    ######################## custom runtime
    runtime = 10*N
    stab_time = N
    # uncomment the following to have runtime in accordance with the lattice size
    # runtime = 10*N*np.log(N)
    ######################## 
    magnitude = main(W,runtime)
    for k in range(0,Nz):
        for j in range(0,Ny):
            for i in range(0,Nx):
                means_run[i,j,k, run] = np.mean(magnitude[i,j,k,stab_time:])
                stds_run[i,j,k, run] = np.std(magnitude[i,j,k,stab_time:])
    # print("done with calculating mean and std for run{}".format(run))
    ################################################################################# EE analysis
    perc_ee[run] = EE_analysis(means_run[:,:,:,run],stds_run[:,:,:,run],runtime,stab_time, W)
means = np.mean(means_run, axis=3)
stds = np.mean(stds_run, axis=3)
print("average percentage of rouge events is: ", np.mean(perc_ee))
print("standard deviation of percentage of rouge events is: ", np.std(perc_ee))

initial position 6 6 8


/var/folders/4g/hvkx9ls12r741x0652yfb8_40000gn/T/ipykernel_9631/1762506082.py:44: ComplexWarning: Casting complex values to real discards the imaginary part
  magnit[i,j,k] += state[i,j,k,l].conjugate()*state[i,j,k,l]


(9, 9, 9)
rouge event count for w = 0.0 is:  52153  out of total runtime*N events, that is 1.0903896721889688 %
initial position 8 7 5
(9, 9, 9)
rouge event count for w = 0.0 is:  52153  out of total runtime*N events, that is 1.0903896721889688 %
initial position 4 1 1
(9, 9, 9)
rouge event count for w = 0.0 is:  52153  out of total runtime*N events, that is 1.0903896721889688 %
initial position 1 0 7
(9, 9, 9)
rouge event count for w = 0.0 is:  52153  out of total runtime*N events, that is 1.0903896721889688 %
initial position 1 8 8
(9, 9, 9)
rouge event count for w = 0.0 is:  52153  out of total runtime*N events, that is 1.0903896721889688 %
initial position 8 4 0
(9, 9, 9)
rouge event count for w = 0.0 is:  52153  out of total runtime*N events, that is 1.0903896721889688 %
initial position 3 2 5
(9, 9, 9)
rouge event count for w = 0.0 is:  52153  out of total runtime*N events, that is 1.0903896721889688 %
initial position 4 8 2
(9, 9, 9)
rouge event count for w = 0.0 is:  52153  out

In [20]:
print(means.shape)
print(stds.shape)
## mean and std deviation, averaged over all the nodes
print("mean of probability amplitudes at each position", np.mean(means.flatten()))
print("std of the means of probability amplitudes at each position", np.std(means.flatten()))
print("std deviation of probability amplitudes at each position", np.mean(stds.flatten()))

## expected value of the probability amplitudes
print("expected value of the probability amplitudes", expected)
print("expected value of the std: ", np.sqrt(expected*(1-expected)))

(9, 9, 9)
(9, 9, 9)
mean of probability amplitudes at each position 0.0013717421124844413
std of the means of probability amplitudes at each position 6.811061460888793e-06
std deviation of probability amplitudes at each position 0.0006071323906988566
expected value of the probability amplitudes 0.0013717421124828531
expected value of the std:  0.0370116256878794


## 1.089704378103705 perc EE when w =0, N = 1331
## 0.85 perc EE when w = 0.3, n = 1331